In [179]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [130]:
titanic_train = pd.read_csv('titanic_data/train.csv')
titanic_train.head(10)
titanic_test_X = pd.read_csv("titanic_data/test.csv")
titanic_test_X.tail(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
408,1300,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.7208,NaN,Q
409,1301,3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S
410,1302,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.7500,NaN,Q
411,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q
412,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [131]:
titanic_train.set_index('PassengerId', inplace=True)
titanic_test_X.set_index('PassengerId', inplace=True)


In [132]:
# sex = {'Sex':{'male': 1, 'female': 0}}
# titanic_train.replace(sex, inplace=True)
titanic_train_X = titanic_train.loc[:, titanic_train.columns != 'Survived']
titanic_train_y = titanic_train['Survived'].reset_index(drop=True)

In [204]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

titanic_train_X['title']=titanic_train_X['Name'].apply(get_title)
titanic_test_X['title']=titanic_test_X['Name'].apply(get_title)
title_lev1=titanic_train_X['title'].unique()
title_lev2=titanic_test_X['title'].unique()
title_lev=list(set().union(title_lev1, title_lev2))

titanic_train_X['title']=pd.Categorical(titanic_train_X['title'], categories=title_lev)
titanic_test_X['title']=pd.Categorical(titanic_test_X['title'], categories=title_lev)
np.sort(titanic_train_X.title.cat.codes.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17],
      dtype=int8)

In [177]:
titanic_train.loc[titanic_train['Cabin'].isna(), 'Survived'].value_counts()

0    481
1    206
Name: Survived, dtype: int64

In [134]:
scalable_columns = ['Age', 'Fare', 'SibSp', 'Parch']
scaler = StandardScaler()
titanic_train_X_scaled = titanic_train_X.copy()
titanic_train_X_scaled[scalable_columns]= scaler.fit_transform(titanic_train_X_scaled[scalable_columns])
titanic_train_X_scaled

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,-0.530377,0.432793,-0.473674,A/5 21171,-0.502445,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.571831,0.432793,-0.473674,PC 17599,0.786845,C85,C
3,3,"Heikkinen, Miss. Laina",female,-0.254825,-0.474545,-0.473674,STON/O2. 3101282,-0.488854,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.365167,0.432793,-0.473674,113803,0.420730,C123,S
5,3,"Allen, Mr. William Henry",male,0.365167,-0.474545,-0.473674,373450,-0.486337,NaN,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,-0.185937,-0.474545,-0.473674,211536,-0.386671,NaN,S
888,1,"Graham, Miss. Margaret Edith",female,-0.737041,-0.474545,-0.473674,112053,-0.044381,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,0.432793,2.008933,W./C. 6607,-0.176263,NaN,S


In [135]:
titanic_test_X_scaled = titanic_test_X.copy()
titanic_test_X_scaled[scalable_columns]= scaler.fit_transform(titanic_test_X_scaled[scalable_columns])

In [136]:
titanic_train_X_scaled = pd.get_dummies(titanic_train_X_scaled, columns=['Sex', 'Embarked'], dtype=bool)

In [137]:
titanic_test_X_scaled = pd.get_dummies(titanic_test_X_scaled, columns=['Sex', 'Embarked'], dtype=bool)

In [138]:
# enc = OneHotEncoder(handle_unknown='ignore')
# titanic_train_scaled = enc.fit_transform(titanic_scaled)
# titanic_test_scaled = enc.fit_transform(titanic_test)
titanic_train_X_scaled

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",-0.530377,0.432793,-0.473674,A/5 21171,-0.502445,NaN,False,True,False,False,True
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.571831,0.432793,-0.473674,PC 17599,0.786845,C85,True,False,True,False,False
3,3,"Heikkinen, Miss. Laina",-0.254825,-0.474545,-0.473674,STON/O2. 3101282,-0.488854,NaN,True,False,False,False,True
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.365167,0.432793,-0.473674,113803,0.420730,C123,True,False,False,False,True
5,3,"Allen, Mr. William Henry",0.365167,-0.474545,-0.473674,373450,-0.486337,NaN,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",-0.185937,-0.474545,-0.473674,211536,-0.386671,NaN,False,True,False,False,True
888,1,"Graham, Miss. Margaret Edith",-0.737041,-0.474545,-0.473674,112053,-0.044381,B42,True,False,False,False,True
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,0.432793,2.008933,W./C. 6607,-0.176263,NaN,True,False,False,False,True


In [139]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score

In [140]:
titanic_train_X_scaled_final = titanic_train_X_scaled.select_dtypes(include=[np.number, bool])

In [141]:
titanic_test_X_scaled_final = titanic_test_X_scaled.select_dtypes(include=[np.number, bool])

In [142]:
titanic_train_X_scaled_final['Age'] = titanic_train_X_scaled_final['Age'].fillna(np.mean(titanic_train_X_scaled_final['Age']))
titanic_train_X_scaled_final['Fare'] = titanic_train_X_scaled_final['Fare'].fillna(np.mean(titanic_train_X_scaled_final['Fare']))

In [143]:
titanic_test_X_scaled_final['Age'] = titanic_test_X_scaled_final['Age'].fillna(np.mean(titanic_test_X_scaled_final['Age']))
titanic_test_X_scaled_final['Fare'] = titanic_test_X_scaled_final['Fare'].fillna(np.mean(titanic_test_X_scaled_final['Fare']))

In [144]:
titanic_test_X_scaled_final.isna().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(titanic_train_X_scaled_final, titanic_train_y, test_size=0.2, random_state=42)

In [174]:
from sklearn.model_selection import GridSearchCV
# rfc = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
# Create a based model
rfc = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rfc, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_
# rfc.fit(X_train, y_train)
# y_pred = rfc.predict(X_test)
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\dcser\anaconda3\envs\DOXA\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 200}